# Title
[]()

In [1]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# iteration 1

In [8]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
from prompts import * # .py file stored in the path above
from db_orm import *
from sources import *
from orm_summarize import *

#########
# Prep: Set parameters
n_choices = 2
article_limit = 2
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=True
folder_path = '/database'


def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    # print('New sources to add to sources table:')
    # print(references_df_dict[iteration_id][['title', 'text','publication', 'year', 'section']])

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table IF NOT ALREADY ADDED
    article_limit = len(references_df_dict[iteration_id]) if article_limit is None else article_limit
    bulk_append(table='sources', input_df=references_df_dict[iteration_id].loc[:article_limit]) # db_orm.py

    # # ##### 
    # # Step 3: Get the new sources for summarization
    # sources_df = get_table(table='sources', limit=article_limit, order='DESC') # db_orm.py

    # if sources_df['title'].tolist() == references_df_dict[iteration_id]['title'].tolist():

    #     # ##### 
    #     # Step 3: Create summaries (functions contained in orm_summarize.py)
    #     chatbot_dict = batch_summarize( # orm_summarize.py
    #         sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
    #         simplify_task, simplify_audience, format_task,
    #         chatbot_dict, temperature=temperature,
    #         system_role=system_role, model=model, max_tokens=1000,
    #         n_choices=n_choices, pause_per_request=pause_per_request,
    #         iteration_id=iteration_id, save_outputs=save_outputs
    #         )
    #     #########
    #     # Step 4: Create summaries table
    #     qna_dict = create_summaries_df(
    #         qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
    #         )

    #     ##########
    #     # Step 5: Add results to summaries and prompts table 
    #     bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    #     return qna_dict[iteration_id]
    return references_df_dict[iteration_id]

qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, article_limit=article_limit)
qna_dict 

Query: SELECT * from gpt_queue ORDER BY id ASC LIMIT 2
Adding 2 rows to the database...
	Trends in Self-Reported Adherence to Healthy Lifestyle Behaviors Among US Adults, 1999 to March 2020
	Lifestyle Enrichment in Later Life and Its Association With Dementia Risk
Error adding data to the database: (psycopg2.ProgrammingError) can't adapt type 'Series'
[SQL: INSERT INTO sources (title, text, abstract, publication, authors, year, month, pub_volume, pub_issue, start_page, end_page, doi, section) SELECT p0::VARCHAR, p1::VARCHAR, p2::VARCHAR, p3::VARCHAR, p4::VARCHAR, p5::INTEGER, p6::VARCHAR, p7::VARCHAR, p ... 546 characters truncated ... , p8, p9, p10, p11, p12, sen_counter) ORDER BY sen_counter RETURNING sources.id, sources.id AS id__1]
[parameters: {'abstract__0': 'Adherence to a healthy lifestyle is associated with lower risks of adverse outcomes. However, trends in multiple lifestyle factors and overall healthy lifestyle status among US adults in recent years are unknown.', 'publicat

,id,title,body,sent_to_sources,section,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text,section
0,0,Trends in Self-Reported Adherence to Healthy L...,Introduction\nAdhering to healthy lifestyle fa...,False,,Adherence to a healthy lifestyle is associated...,JAMA network open,"Yue Li, Peng-Fei Xia, Ting-Ting Geng, Zhou-Zhe...",2023,,6,7,e2323584,,10.1001/jamanetworkopen.2023.23584,Introduction\nAdhering to healthy lifestyle fa...,
1,24,Lifestyle Enrichment in Later Life and Its Ass...,"Introduction\nIn 2022, there were 55 million i...",False,,Lifestyles enriched with socially and mentally...,JAMA network open,"Zimu Wu, Danushika H Pandigama, Jo Wriggleswor...",2023,,6,7,e2323690,,10.1001/jamanetworkopen.2023.23690,"Introduction\nIn 2022, there were 55 million i...",


In [13]:
for column, value in qna_dict.loc[0].items():
    print(column, type(value))

id <class 'numpy.int64'>
title <class 'str'>
body <class 'str'>
sent_to_sources <class 'numpy.bool_'>
section <class 'str'>
abstract <class 'str'>
publication <class 'str'>
authors <class 'str'>
year <class 'str'>
month <class 'str'>
pub_volume <class 'str'>
pub_issue <class 'str'>
start_page <class 'str'>
end_page <class 'str'>
doi <class 'str'>
text <class 'str'>
section <class 'str'>


In [14]:
qna_dict['id'] = qna_dict['id'].astype(int)
for column, value in qna_dict.loc[0].items():
    print(column, type(value))

id <class 'numpy.int32'>
title <class 'str'>
body <class 'str'>
sent_to_sources <class 'numpy.bool_'>
section <class 'str'>
abstract <class 'str'>
publication <class 'str'>
authors <class 'str'>
year <class 'str'>
month <class 'str'>
pub_volume <class 'str'>
pub_issue <class 'str'>
start_page <class 'str'>
end_page <class 'str'>
doi <class 'str'>
text <class 'str'>
section <class 'str'>


In [15]:
type(9)

int

In [16]:
text_df = get_table(table='gpt_queue', limit=article_limit)
for column, value in text_df.loc[0].items():
    print(column, type(value))


Query: SELECT * from gpt_queue ORDER BY id ASC LIMIT 2
id <class 'numpy.int64'>
title <class 'str'>
body <class 'str'>
sent_to_sources <class 'numpy.bool_'>
section <class 'str'>


In [17]:
sources = get_table(table='sources', limit=article_limit)
for column, value in sources.loc[0].items():
    print(column, type(value))

Query: SELECT * from sources ORDER BY id ASC LIMIT 2
id <class 'numpy.int64'>
title <class 'str'>
text <class 'str'>
abstract <class 'str'>
publication <class 'str'>
authors <class 'str'>
year <class 'numpy.int64'>
month <class 'str'>
pub_volume <class 'str'>
pub_issue <class 'str'>
start_page <class 'str'>
end_page <class 'str'>
doi <class 'str'>
section <class 'NoneType'>
excess_tokens <class 'NoneType'>


In [20]:
type(int(sources.loc[0, 'id']))

int

In [23]:
type(int(qna_dict.loc[0, 'sent_to_sources']))

int

# 2 able to re-add data to gpt_queue table

In [26]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
from prompts import * # .py file stored in the path above
from db_orm import *
from sources import *
from orm_summarize import *

#########
# Prep: Set parameters
n_choices = 2
article_limit = 2
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=True
folder_path = '/database'


def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    # print('New sources to add to sources table:')
    # print(references_df_dict[iteration_id][['title', 'text','publication', 'year', 'section']])

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table IF NOT ALREADY ADDED
    article_limit = len(references_df_dict[iteration_id]) if article_limit is None else article_limit
    bulk_append(table='gpt_queue', input_df=text_df) # db_orm.py

    # # ##### 
    # # Step 3: Get the new sources for summarization
    # sources_df = get_table(table='sources', limit=article_limit, order='DESC') # db_orm.py

    # if sources_df['title'].tolist() == references_df_dict[iteration_id]['title'].tolist():

    #     # ##### 
    #     # Step 3: Create summaries (functions contained in orm_summarize.py)
    #     chatbot_dict = batch_summarize( # orm_summarize.py
    #         sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
    #         simplify_task, simplify_audience, format_task,
    #         chatbot_dict, temperature=temperature,
    #         system_role=system_role, model=model, max_tokens=1000,
    #         n_choices=n_choices, pause_per_request=pause_per_request,
    #         iteration_id=iteration_id, save_outputs=save_outputs
    #         )
    #     #########
    #     # Step 4: Create summaries table
    #     qna_dict = create_summaries_df(
    #         qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
    #         )

    #     ##########
    #     # Step 5: Add results to summaries and prompts table 
    #     bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    #     return qna_dict[iteration_id]
    return references_df_dict[iteration_id]

qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, article_limit=article_limit)
qna_dict 

Query: SELECT * from gpt_queue ORDER BY id ASC LIMIT 2
Adding 2 rows to the database...
	Trends in Self-Reported Adherence to Healthy Lifestyle Behaviors Among US Adults, 1999 to March 2020
	Lifestyle Enrichment in Later Life and Its Association With Dementia Risk
Data added successfully!


,id,title,body,sent_to_sources,section,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text,section
0,0,Trends in Self-Reported Adherence to Healthy L...,Introduction\nAdhering to healthy lifestyle fa...,False,,Adherence to a healthy lifestyle is associated...,JAMA network open,"Yue Li, Peng-Fei Xia, Ting-Ting Geng, Zhou-Zhe...",2023,,6,7,e2323584,,10.1001/jamanetworkopen.2023.23584,Introduction\nAdhering to healthy lifestyle fa...,
1,24,Lifestyle Enrichment in Later Life and Its Ass...,"Introduction\nIn 2022, there were 55 million i...",False,,Lifestyles enriched with socially and mentally...,JAMA network open,"Zimu Wu, Danushika H Pandigama, Jo Wriggleswor...",2023,,6,7,e2323690,,10.1001/jamanetworkopen.2023.23690,"Introduction\nIn 2022, there were 55 million i...",


In [27]:
text_df 

,id,title,body,sent_to_sources,section
0,0,Trends in Self-Reported Adherence to Healthy L...,Introduction\nAdhering to healthy lifestyle fa...,False,
1,24,Lifestyle Enrichment in Later Life and Its Ass...,"Introduction\nIn 2022, there were 55 million i...",False,


In [28]:
for column, value in text_df.loc[0].items():
    print(column, type(value))

id <class 'numpy.int64'>
title <class 'str'>
body <class 'str'>
sent_to_sources <class 'numpy.bool_'>
section <class 'str'>


In [32]:
to_append = qna_dict.loc[0:0]
for column in to_append.columns[1:]:
    to_append.loc[0, column] = 'test'
to_append

,id,title,body,sent_to_sources,section,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text,section
0,0,test,test,test,test,test,test,test,test,test,test,test,test,test,test,test,test


In [33]:
bulk_append(table='sources', input_df=to_append)

Adding 1 rows to the database...
	test
Error adding data to the database: (psycopg2.ProgrammingError) can't adapt type 'Series'
[SQL: INSERT INTO sources (title, text, abstract, publication, authors, year, month, pub_volume, pub_issue, start_page, end_page, doi, section) VALUES (%(title)s, %(text)s, %(abstract)s, %(publication)s, %(authors)s, %(year)s, %(month)s, %(pub_volume)s, %(pub_issue)s, %(start_page)s, %(end_page)s, %(doi)s, %(section)s) RETURNING sources.id]
[parameters: {'title': 'test', 'text': 'test', 'abstract': 'test', 'publication': 'test', 'authors': 'test', 'year': 'test', 'month': 'test', 'pub_volume': 'test', 'pub_issue': 'test', 'start_page': 'test', 'end_page': 'test', 'doi': 'test', 'section': section    test
section    test
Name: 0, dtype: object}]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [34]:
to_append2 = to_append.drop(columns=['id'])
to_append2

,title,body,sent_to_sources,section,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text,section
0,test,test,test,test,test,test,test,test,test,test,test,test,test,test,test,test


In [35]:
bulk_append(table='sources', input_df=to_append2)

Adding 1 rows to the database...
	test
Error adding data to the database: (psycopg2.ProgrammingError) can't adapt type 'Series'
[SQL: INSERT INTO sources (title, text, abstract, publication, authors, year, month, pub_volume, pub_issue, start_page, end_page, doi, section) VALUES (%(title)s, %(text)s, %(abstract)s, %(publication)s, %(authors)s, %(year)s, %(month)s, %(pub_volume)s, %(pub_issue)s, %(start_page)s, %(end_page)s, %(doi)s, %(section)s) RETURNING sources.id]
[parameters: {'title': 'test', 'text': 'test', 'abstract': 'test', 'publication': 'test', 'authors': 'test', 'year': 'test', 'month': 'test', 'pub_volume': 'test', 'pub_issue': 'test', 'start_page': 'test', 'end_page': 'test', 'doi': 'test', 'section': section    test
section    test
Name: 0, dtype: object}]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [36]:
to_append2.loc[0, 'section']

section    test
section    test
Name: 0, dtype: object

# *End of Page*